In [2]:
%reload_ext autoreload
%autoreload 2
import os
import shutil
import sys
import numpy as np
import pandas as pd
import nglview as nv
import mdtraj as md
from sstmap.site_water_analysis import SiteWaterAnalysis
from sstmap.utils import write_watpdb_from_coords
from insightdemo.utils import format_bytes

# What do drug targets look like?

In [3]:
view = nv.show_pdbid("1FJS")
view.add_surface(selection="protein", color="blue")
# parameters for the NGL stage object
view.parameters = {
    # "percentages, "dist" is distance too camera in Angstrom
    "clipNear": 0, "clipFar": 100, "clipDist": 10,
    # percentages, start of fog and where on full effect
    "fogNear": 0, "fogFar": 100,
    # background color
    "backgroundColor": "black",
}
view

NGLWidget()

# Simulating Proteins

In [4]:
traj = md.load("data/test.trr", top="data/aa2ar.pdb")
view = nv.show_mdtraj(traj)
view.add_licorice('water and not hydrogen')
view

NGLWidget(count=100)

# Data inside simulations

In [14]:
print(traj)
# a more realistics trajectory would contain 100,000 frames
a_float = np.float64(-1.5)
typical_n_frame = 100000
typical_particle_size = 70000
dims = 3
data_size = a_float.nbytes * typical_n_frame * typical_particle_size * dims
print(format_bytes(data_size))

<mdtraj.Trajectory with 100 frames, 65358 atoms, 15489 residues, and unitcells>
156.46 GB


# Too much complexity? Let's try SSTMap

In [5]:
curr_dir = os.path.abspath("data/")
data_dir = os.path.join(curr_dir, "SSTMap_HSA")

if not os.path.exists(data_dir):
    os.makedirs(data_dir)
else:
    shutil.rmtree(data_dir)
    os.makedirs(data_dir)

top = os.path.join(curr_dir, "aa2ar.prmtop")
traj = os.path.join(curr_dir, "md100ps.nc")
start_frame = 0
num_frames = 100
ligand = os.path.abspath("data/ligand.pdb")
os.chdir(data_dir)
h = SiteWaterAnalysis(top, traj,
                    start_frame=start_frame, num_frames=num_frames,
                    ligand_file=ligand, prefix="aa2ar")
h.initialize_hydration_sites()
h.print_system_summary()
h.calculate_site_quantities()
h.write_calculation_summary()
os.chdir(curr_dir)

Initializing ...
Obtaining non-bonded parameters for the system ...
Total time running generate_nonbonded_params: 17.38 seconds
Done.
Assigning hydrogen bond types ...
Total time running assign_hb_types: 5.74 seconds
Done.
Total time running __init__: 36.29 seconds
Reading trajectory for clustering.
Performing an initial clustering over 10 frames.
Reading trajectory to obtain water molecules for each cluster.
Refining initial cluster positions by considering 100 frames.
Final number of clusters: 33
Total time running generate_clusters: 24.01 seconds
Total time running initialize_site_data: 0.00 seconds
Total time running initialize_hydration_sites: 24.01 seconds
System information:
	Parameter file: /Users/kamranhaider/Dropbox/insightdemo/data/aa2ar.prmtop

	Trajectory: /Users/kamranhaider/Dropbox/insightdemo/data/md100ps.nc

	Total Atoms: 65358, Waters: 15186, Solute Atoms: 4614

	Number of clusters: 33

Writing PDB file containing all HSA region water molecules for entropy calculation

In [9]:
os.chdir("/Users/kamranhaider/Dropbox/insightdemo/")
result = md.load_pdb("data/aa2ar_prot_map.pdb")
view = nv.show_mdtraj(result)
view.add_ball_and_stick('water and not hydrogen')
view

NGLWidget()

In [27]:
!head data/SSTMap_HSA/aa2ar_hsa_summary.txt

index x y z nwat occupancy Esw EswLJ EswElec Eww EwwLJ EwwElec Etot Ewwnbr TSsw_trans TSsw_orient TStot Nnbrs Nhbww Nhbsw Nhbtot f_hb_ww f_enc Acc_ww Don_ww Acc_sw Don_sw solute_acceptors solute_donors
0 38.37 43.98 54.62 97 0.97 -10.897226 0.449734 -11.346960 -0.986011 0.051487 -1.037497 -11.883237 -0.870463 -0.279239 -2.994422 -3.273662 0.268041 0.134021 3.350515 3.484536 0.134021 0.000000 0.000000 0.134021 1.783505 1.567010 MET170-O,ASN174-OD1 HIS232-ND1,ASN235-ND2
1 33.81 34.43 63.48 88 0.88 -1.536784 -0.775240 -0.761544 -11.566633 2.274721 -13.841354 -13.103417 -2.396671 -0.864498 -2.836987 -3.701485 3.727273 3.204545 0.875000 4.079545 0.878788 0.000000 1.250000 1.954545 0.875000 0.000000  PHE161-N
2 30.77 31.88 58.39 86 0.86 -5.226516 -1.095290 -4.131225 -5.470104 0.757426 -6.227531 -10.696620 -2.777529 -1.287546 -1.867580 -3.155125 1.558140 1.546512 1.127907 2.674419 0.994186 0.000000 1.046512 0.500000 0.000000 1.127907 TYR7-OH,ALA61-O 
3 34.44 36.96 64.73 91 0.91 -6.666751 -0.9